In [1]:
%matplotlib inline
import torch
import torchvision
from torch import nn
import pandas
import os
import random
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
from IPython import display
import time
import shutil
import matplotlib_inline.backend_inline

/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [3]:
path=os.path.join(os.getcwd(),"dataset")

In [4]:
def splitData( path,train_r=0.7,test_r=0.2,val_r=0.1 ):
# rename those files with JPG extension
    path=os.path.join(os.getcwd(),"dataset")
    f=[ f for f in os.listdir(path) if not f.startwith(".")]

    for fname in f:
        if ".ipy" in fname:
            print(fname)
        pre , ext = os.path.splitext(fname)
        if ext != ".jpg":
            os.rename(os.path.join(path , fname) , os.path.join(path,pre) + ".jpg")
# shuffle dataset and get train,test,val dataset     
    shuffle=os.listdir(path)
    shuffle.remove(".ipynb_checkpoints")
    shuffle.remove(".ipynb_checkpoints.jpg")
    random.shuffle(shuffle)
    l=len(shuffle)
    train=shuffle[:int(l * train_r)]
    test=shuffle[len(train): int(l * (train_r + test_r))]
    val=shuffle[(len(train) + len(test)):]
    print("train: " , len(train) , "ratio: " , len(train)/l)
    print("test: " , len(test) , "ratio: " , len(test)/l)
    print("val: " , len(val) , "ratio: " , len(val)/l)
    return train,test,val

In [10]:
def getDataset(train,test,val):
#  create 4 dataframes(train , test , val , total) for data analysis
    total=pandas.read_csv(os.path.join(os.getcwd() ,  "info_images.txt") , sep="\t")
    for fname in train:
        total.loc[total["FileName"]==fname, "D_type"]="train"
    for fname in test:
        total.loc[total["FileName"]==fname, "D_type"]="test"
    for fname in val:
        total.loc[total["FileName"]==fname, "D_type"]="val"
    oneHot=pandas.get_dummies(total["D_type"] , prefix="Type")
    trainDF=total.loc[oneHot["Type_train"] != 0]
    testDF=total.loc[oneHot["Type_test"] != 0]
    valDF=total.loc[oneHot["Type_val"] != 0]
    trainDF.to_csv("trainDF.csv")
    testDF.to_csv("testDF.csv")
    valDF.to_csv("valDF.csv")
    # print(trainDF , testDF , valDF)
    return trainDF , testDF , valDF , total

In [11]:
def dupFiles (SrCpath,train,tes,val):
    DesTpath=os.path.join(os.getcwd(),"data")
    os.makedirs(DesTpath, exist_ok=True)
    d_dict={"train":[os.path.join(DesTpath , "train") , train], "test":[os.path.join(DesTpath , "test") , test], "val":[os.path.join(DesTpath , "val") , val]}
    for _,d in d_dict.items():
        print(_)
        p=d[0]
        if os.path.exists(p) and os.path.isdir(p):
            shutil.rmtree(p)
        os.makedirs(p)
        Flist=d[1]
        for fname in Flist:
            # print(fname)
            shutil.copy(os.path.join(SrCpath , fname), p)
    

In [12]:
train,test,val = splitData(path)
trainDF,testDF,valDF,total = getDataset(train,test,val)

train:  2121 ratio:  0.7
test:  605 ratio:  0.19966996699669967
val:  304 ratio:  0.10033003300330033


In [13]:
dupFiles (path , train , test , val)

train
test
val


In [9]:
train_ag=trainDF[["FileName","elo_reduced"]]
test_ag=testDF[["FileName","elo_reduced"]]
val_ag=valDF[["FileName","elo_reduced"]]
df_ls=[train_ag,test_ag,val_ag]
for df in df_ls:
    df.rename(columns={"FileName":"image"},inplace=True)
    df["elo_reduced"]= np.around(df["elo_reduced"],decimals=1)
train_ag.to_csv("train_ag.csv")
test_ag.to_csv("test_ag.csv")
val_ag.to_csv("val_ag.csv")

/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
ts=pandas.read_csv("trueskill.csv")
ts["rank_c"]=pandas.qcut(ts["rank"],q=100,labels=[i for i in range(100)])

In [18]:
train_ag=trainDF[["FileName"]]
train_ag["rank"]=0
test_ag=testDF[["FileName"]]
test_ag["rank"]=0
val_ag=valDF[["FileName"]]
val_ag["rank"]=0
for fname in train:
    if len(ts.loc[ts["Name"]==fname.lower()])!=1:
        print(fname.lower())
    train_ag.loc[train_ag["FileName"]==fname,"rank"]=ts.loc[ts["Name"]==fname.lower(),"rank_c"].item()
for fname in test:
    test_ag.loc[test_ag["FileName"]==fname,"rank"]=ts.loc[ts["Name"]==fname.lower(),"rank_c"].item()
for fname in val:
    val_ag.loc[val_ag["FileName"]==fname,"rank"]=ts.loc[ts["Name"]==fname.lower(),"rank_c"].item()
df_ls=[train_ag,test_ag,val_ag]
for df in df_ls:
    df.rename(columns={"FileName":"image"},inplace=True)
    # df["rank"]= np.around(df["rank"],decimals=1)
# train_ag=pandas.concat([train_ag,test_ag])
train_ag.to_csv("train_ag2.csv")
test_ag.to_csv("test_ag2.csv")
val_ag.to_csv("val_ag2.csv")


/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/userhome/cs/chan0305/ENTER/envs/walkability/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [171]:
total.describe()[["mean_elo_rating","st_elo_rating","elo_reduced"]]

,mean_elo_rating,st_elo_rating,elo_reduced
count,3030.000000,3030.000000,3.030000e+03
mean,1000.000000,64.206672,-1.407015e-17
std,198.078446,16.888118,1.000000e+00
min,456.916667,13.784873,-2.741759e+00
25%,858.125000,52.258463,-7.162566e-01
50%,1003.083333,63.436917,1.556622e-02
75%,1147.083333,75.525319,7.425509e-01
max,1534.833333,127.706695,2.700109e+00
